In [1]:
# Python 3 server example
from http.server import BaseHTTPRequestHandler, HTTPServer
from io import BytesIO
from tempfile import SpooledTemporaryFile
from cgi import FieldStorage
import sys, os
from PIL import Image 
import threading
import time
import matplotlib.pyplot as plt
import io
import json
import urllib 
import base64
import random
import numpy as np
import torch

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [2]:
hostName = "0.0.0.0"
serverPort = 8080
run_thread= None
model_file='Weapons.pt'

In [3]:
import ipywidgets
import traitlets
name_widget = ipywidgets.Text(description='model',value=model_file)
prediction_widget = ipywidgets.Valid(value=False, description='Predict',)

live_execution_widget = ipywidgets.VBox([
    name_widget,
    prediction_widget
])

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load(model_file)
model.eval()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:604: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)


RecursiveScriptModule(
  original_name=Sequential
  (0): RecursiveScriptModule(original_name=Linear)
  (1): RecursiveScriptModule(original_name=BatchNorm1d)
  (2): RecursiveScriptModule(original_name=Dropout1d)
  (3): RecursiveScriptModule(original_name=ReLU)
  (4): RecursiveScriptModule(original_name=Linear)
  (5): RecursiveScriptModule(original_name=BatchNorm1d)
  (6): RecursiveScriptModule(original_name=Dropout1d)
  (7): RecursiveScriptModule(original_name=ReLU)
  (8): RecursiveScriptModule(original_name=Linear)
)

In [5]:
# model.to(device)

In [6]:
class MyServer(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(bytes("<html><head><title>https://pythonbasics.org</title></head>", "utf-8"))
        self.wfile.write(bytes("<body>", "utf-8"))
        self.wfile.write(bytes("<p>This is a body segmentation web server.</p>", "utf-8"))
        self.wfile.write(bytes("</body></html>", "utf-8"))


    def do_POST(self):
        length = int(self.headers['Content-Length'])
        buffer = self.rfile.read(length)
        data=urllib.parse.parse_qs(buffer.decode())
        enc = json.loads(data["tensor"][0])
        dataType = np.dtype(enc[0])
        dataArray = np.frombuffer(base64.decodestring(bytes(enc[1], encoding="raw_unicode_escape")), dataType)
        if len(enc) > 2:
             dataArray.reshape(enc[2]) 
        preprocessed=torch.from_numpy(dataArray)
        since = time.time()
        preprocessed =preprocessed.reshape(1,512)
        # print(preprocessed)
        output = model(preprocessed)
        output = output.detach().cpu() 
        # print(output)
        output = torch.argmax(output, 1)
        # print(output)
        time_elapsed = time.time() - since
        prediction_widget.value= (output.item()==1)
        # print("{} {}s".format(str(output.item()),str(time_elapsed)))

In [7]:
# tensor1=torch.ones(512)
# print(tensor1)
# tensor2=torch.ones(1,512)
# print(tensor2)
# print(tensor1.reshape(1,512))

In [8]:
display(live_execution_widget)

In [9]:
webServer = HTTPServer((hostName, serverPort), MyServer)
print("Server started http://%s:%s" % (hostName, serverPort))

try:
    webServer.serve_forever()
except KeyboardInterrupt:
    pass


Server started http://0.0.0.0:8080


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: decodestring() is a deprecated alias since Python 3.1, use decodebytes()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /media/nvidia/NVME/pytorch/pytorch-v1.10.0/torch/csrc/utils/tensor_numpy.cpp:189.)


In [10]:
webServer.server_close()